# Mundo 5 - Selenium e composição do IBOV

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
driver.get('''https://www.b3.com.br/pt_br/market-data-e-indices/indices
/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm''')

In [ ]:
driver.maximize_window()

In [ ]:
boto_download_planilha = driver.find_element("xpath", '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a')

#nao vai achar porque ta dentro de um iframe. toma muito cuidado com isso!!! 

# Existem duas formas de trocar pra outra URL:

* Achar o iframe e trocar com o selenium
* Utilizar um recurso de espera e trocar automaticamente

# Modo 1:

In [ ]:
iframe = driver.find_element("id", "bvmf_iframe")

driver.switch_to.frame(iframe)

# Modo 2:

Esse modo serve pra 1 milhão de coisas e é mais genérico

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.ID, "bvmf_iframe")))

#element_to_be_clickable
#text_to_be_present_in_element

In [ ]:
planilha = driver.find_element("xpath", '//*[@id="divContainerIframeB3"]/div/div[1]/form/div[2]/div/div[2]/div/div/div[1]/div[2]/p/a')

In [ ]:
#planilha.click()

driver.execute_script("arguments[0].click();", planilha)

# Como selecionar caixinhas com o Selenium

In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
botao_de_selecao = driver.find_element("id", "selectPage")

caixa_de_selecao = Select(botao_de_selecao).select_by_visible_text("60")

In [ ]:
driver.quit() 
#cuidado com esse quit antes de baixar

In [ ]:
import pandas as pd
from datetime import date

In [ ]:
dia = date.today().day
mes = date.today().month
ano = date.today().year

if dia < 10:
    
    dia = "0" + str(dia)

if mes < 10:
    
    mes = "0" + str(mes)
    
ano = str(ano)[2:]

print(dia, mes, ano)

In [ ]:
ibovespa_comp = pd.read_csv(fr"C:\Users\lsiqu\Downloads\IBOVDia_{dia}-{mes}-{ano}.csv", sep = ";", 
                            skipfooter = 2, encoding = 'ISO-8859-1', engine = 'python', decimal = ',',
                           thousands = ".", header = 1, index_col = False)

ibovespa_comp

# Exercícios 

Exercício 144: Faça webscraping do dólar no banco central, pegando as cotações do dólar com a data já pré prenchida no formulário.

Link: https://www.bcb.gov.br/estabilidadefinanceira/historicocotacoes

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.bcb.gov.br/estabilidadefinanceira/historicocotacoes'

driver.get(url)

WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "/html/body/app-root/app-root/main/dynamic-comp/div/div[2]/div[1]/div/iframe")))

botao_pesquisar = driver.find_element("xpath", '/html/body/div/form/div/input')

driver.execute_script("arguments[0].click();", botao_pesquisar)

tabela = driver.find_element("xpath", '/html/body/div/table')

html_tabela = tabela.get_attribute('outerHTML')

cotacao_dolar = pd.read_html(html_tabela)[0]

driver.quit()

cotacao_dolar

In [ ]:
cotacao_dolar = cotacao_dolar.droplevel(level = 0, axis = 1)
cotacao_dolar = cotacao_dolar.dropna(axis = 1).drop(18, axis = 0)

cotacao_dolar